# Solution
Here is the solution I computed. The image below shows the drone image stitched with the base map.  

![WarpedImage](assets/warped.png)  

The position of drone relative to the base map is X: [175.46782683] Y: [-51.13010725] Z: [0.85639727] pixels.  
This translation is upto a scale factor.  
The rotation with respect to the base map is [261.62]  [-0.0208]  [0.0151] following the ZYX euler angle convention.
The positive anle being the counter clockwise direction

# Explanation
In this section I discuss how I approached this problem.  
## Initial thoughts  
These were my initial thoughts as saw the problem.  
Motion between two monocular images can be described by either a fundamental matrix or a homography.  
Fundamental matrix is better when the scene is non planar.  
Homography is better when the scene is planar.  
Since the drone is at an altitude of 119m, the scene can reasonably be assumed to be planar.  
Also satellite imagery is taken at a very high altitude, it can also be assumed to be planar.  
I would have to compute keypoints between images, match them, compute the homography and decompose  
it to get the motion between two scenes.

# First approach
Generally, I approach a problem starting forom ground up  
So I collected some data from google earth which was simpler as compared to the given task  
I then setup the pipeline using SIFT  
It worked and i was able to get good stiching between images.  
But it didn't work for the given frames which i already suspected.  
SIFT while being invariant to scale and rotations, is not very robust against illumination changes.  
Both images had keypoints, but there were not enough good matches to compute a good homography.  
This resulted in a poor stitching if not a stitching at all.  
I tried the same pipeline with ORB and AKAZE as well.  
I have included this pipeline at the end of this notebook.  
# Second approach
The next approach was to look for a feature descriptor which is robust against illumination changes  
and provides descriptors which are kind of time invariant.  
Learning based feature detectors are known to outperform hand crafted feature detectors  
After reviewing some literature, I chose SuperPoint as the keypoint detector and descriptor and SuperGlue as the matching algorithm.  
I am using the model weights provided by the authors.  
Below is my implementation of a basic visual navigation system. Comments are added where deemed necessary.   

In [1]:
# Necessary imports
import numpy as np
import matplotlib.cm as cm
import torch
import cv2
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation as R
from models.matching import Matching
from models.utils import (make_matching_plot,read_image)

# Turning off gradient calculation since we are in inference mode
torch.set_grad_enabled(False)

In [2]:
# Configurations for SuperPoint and SuperGlue models.
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Running inference on device \"{}\"'.format(device))
config = {
    'superpoint': {
        'nms_radius': 4,
        'keypoint_threshold': 0.005,
        'max_keypoints': 1024
    },
    'superglue': {
        'weights': 'outdoor',
        'sinkhorn_iterations': 30,
        'match_threshold': 0.2,
    }
}

matching = Matching(config).eval().to(device)
# Image paths
baseMap = 'assets/lendurai/base_map.jpg'
dronePhoto = 'assets/lendurai/drone_photo.jpg'

# Lists to store required data
heuristic = []
goodKp1 = []
goodKp2 = []
images = []


Running inference on device "cuda"
Loaded SuperPoint model
Loaded SuperGlue model ("outdoor" weights)


Heuristic is chosen based on the confidence of the number of keypoints  
This confidence value is provided by the matching algorithm  
Those keypoints are chosen which have the highest confidence sum  

While SuperGlue provided good matches, it didn't perform well under higher rotations  
My approach was to rotate the drone image by a known angle, compute the matches and then account for this rotation  
while decomposing the homography  
I rotate the image by known rotations and select the image and keypoints with the highest value of the heuristic  
These keypoints are then used to compute the homography  
For homography computation, points should be spread over the whole image  
otherwise the area having most of the points will have a good warp as compared to other region   

In [3]:
# Load the base image.
image0, inp0, scales0 = read_image(
            baseMap, device, [-1], 0, False)

for i in range(0, 4, 1):
    image1, inp1, scales1 = read_image(
            dronePhoto, device, [-1], i, False)
    # Perform the matching.
    pred = matching({'image0': inp0, 'image1': inp1})
    pred = {k: v[0].cpu().numpy() for k, v in pred.items()}
    # Get the matches
    kpts0, kpts1 = pred['keypoints0'], pred['keypoints1']
    matches, conf = pred['matches0'], pred['matching_scores0']

    # Select the valid matches, keypoints which donot have a match have -1
    valid = matches > -1
    mkpts0 = kpts0[valid]
    mkpts1 = kpts1[matches[valid]]
    mconf = conf[valid]

    # Save visualization
    color = cm.jet(mconf)
    viz_path = 'assets/viz/angle_{}.png'.format(i*90)
    text = [
        'SuperGlue',
        'Keypoints: {}:{}'.format(len(kpts0), len(kpts1)),
        'Matches: {}'.format(len(mkpts0)),
        'Rotation: {}'.format(i*90)
    ]

    small_text = [
                    'Matching using Superglue'
                ]
    
    make_matching_plot(
                    image0, image1, kpts0, kpts1, mkpts0, mkpts1, color,
                    text, viz_path, False,
                    False, False, 'Matches', small_text)
    
    # Choosing the matches with confidence above 0.72
    goodMatches = mconf > 0.72
    # Store all the good matches, heuristic and images
    goodKp1.append(mkpts0[goodMatches])
    goodKp2.append(mkpts1[goodMatches])
    heuristic.append(sum(mconf[goodMatches]))
    images.append(image1)

# Select the matches with the highest heuristic and calculate the homography
bestmatchIndex = heuristic.index(max(heuristic))
H, _ = cv2.findHomography(np.array(goodKp2[bestmatchIndex]), np.array(goodKp1[bestmatchIndex]), cv2.RANSAC, 4)

Homography decomposition gives 4 solutions, 2 can be discarded because they have negative value of z translation  
which means our scene is behind the camera. The third solution is discarded because rotation about the x axis is greater than  
100 degrees and since we know that our camera is looking downwards, that solution is also not possible
So we are left with one solution.

I warp and stitch the image for better visualization  

In [4]:
# Decompose the homography matrix to get the rotation and translation
num, Rs, Ts, Ns = cv2.decomposeHomographyMat(H, np.eye(3))

# Convert rotation matrices to euler angles using skikit
r = R.from_matrix(Rs)
angles = r.as_euler('zyx', degrees=True)
for angles, translation in zip(angles, Ts):
    if translation[2] > 0 and angles[2] < 135:
        print("Angles: ", angles)
        print("Translation: ", translation[0], translation[1], translation[2])

# Warp the drone photo
# These images are loaded independent of the implemented function
baseMapOrig = cv2.imread(baseMap, cv2.IMREAD_GRAYSCALE)

h,w = images[bestmatchIndex].shape
pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
dst = cv2.perspectiveTransform(pts,H)

height, width = baseMapOrig.shape
dronePhotoWarped = cv2.warpPerspective(images[bestmatchIndex], H, (width, height))

# Replcae the base map with the warped drone photo
baseMapStiched = baseMapOrig.copy()
baseMapStiched[dronePhotoWarped > 0] = dronePhotoWarped[dronePhotoWarped > 0]

baseMapStiched = cv2.polylines(baseMapStiched,[np.int32(dst)],True,0,2, cv2.LINE_AA)

# Display the warped drone photo
plt.imshow(baseMapStiched, cmap='gray')
plt.savefig('assets/warped1.png')
plt.close()


Angles:  [-8.3845048  -0.02081877  0.01513642]
Translation:  [175.46782683] [-51.13010725] [0.85639727]


# Practical applications
I can image a complete system for drone localization, where drone position is estimated using visual odometry.  
This approach could be used to get the global position of the drone if we have gps aligned map images  
This could also be used in an EKF for better position estimates  
To have a robust solution, drone images could be north aligned using the onboard IMU, if it provides compass headings.  
This is done because the map data will be north aligned and we won't have to rotate the images from the drone and follow a heuristic based approach  
as demonstrated above.  

# Hand crafted features pipeline
The section below contains the pipeline developed using hand crafted features.

In [ ]:
# Necessary imports
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation

In [ ]:
# # Load Images
baseMap  = cv2.imread('assets/lendurai/fullMapold.png', cv2.IMREAD_GRAYSCALE)
droneImage = cv2.imread('assets/lendurai/droneRotated.png', cv2.IMREAD_GRAYSCALE)

# Resize Map
baseMap = cv2.resize(baseMap, (300, 200))

# Display Images
fig, ax = plt.subplots(1, 2)
ax[0].imshow(baseMap, cmap='gray')
ax[0].set_title('Base Map')
ax[1].imshow(droneImage, cmap='gray')
ax[1].set_title('Drone Image')
plt.show()

In [ ]:
# Choose the detector, SIFT performs the best but others can also be fine tuned
detector_name = 'SIFT'
detector = cv2.SIFT_create()
if detector_name == 'ORB':
    detector = cv2.ORB_create()
elif detector_name == 'SIFT':
    detector = cv2.SIFT_create()
elif detector_name == 'AKAZE':
    detector = cv2.AKAZE_create()


In [ ]:
# Find the keypoints and descriptors with SIFT
mapKeypoints, mapDescriptors = detector.detectAndCompute(baseMap, None)
droneKeypoints, droneDescriptors = detector.detectAndCompute(droneImage, None)

# Draw Keypoints
mapKeypointsImage = cv2.drawKeypoints(baseMap, mapKeypoints, None)
droneKeypointsImage = cv2.drawKeypoints(droneImage, droneKeypoints, None)

# Display Images
fig, ax = plt.subplots(1, 2)
ax[0].imshow(mapKeypointsImage)
ax[0].set_title('Map Keypoints')
ax[1].imshow(droneKeypointsImage)
ax[1].set_title('Drone Keypoints')
plt.show()

In [ ]:
# Different metrics are used for different detectors
# For sift euclidean distance is used
# For ORB and AKAZE hamming distance is used
if detector_name == 'ORB' or detector_name == 'AKAZE':
    matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = matcher.match(droneDescriptors, mapDescriptors)
    matches = sorted(matches, key=lambda x: x.distance)
    goodMatches = matches[:10]
elif detector_name == 'SIFT':
    # Parameters for Flann
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks = 50)

    # Initialize Flann
    matcher = cv2.FlannBasedMatcher(index_params, search_params)
    # Find the matches
    matches = matcher.knnMatch(droneDescriptors, mapDescriptors, k=2)
    # Store all the good matches as per Lowe's ratio test.
    goodMatches = []
    for m,n in matches:
        if m.distance < 0.7*n.distance:
            goodMatches.append(m)
else:
    matcher = cv2.BFMatcher()

In [ ]:
# Draw Matches
matchesImage = cv2.drawMatches(droneImage, droneKeypoints, baseMap, mapKeypoints, goodMatches, None)

# Display Image
plt.imshow(matchesImage)

In [ ]:
# get the points from the good matches
src_pts = np.float32([ droneKeypoints[m.queryIdx].pt for m in goodMatches ]).reshape(-1,1,2)
dst_pts = np.float32([ mapKeypoints[m.trainIdx].pt for m in goodMatches ]).reshape(-1,1,2)

# Find the homography matrix 
M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,2.0)
matchesMask = mask.ravel().tolist()

# Add lines for visualization 
h,w = droneImage.shape
pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
dst = cv2.perspectiveTransform(pts,M)

# Warp the drone photo
height, width = baseMap.shape
dronePhotoWarped = cv2.warpPerspective(droneImage, M, (width, height))

# Replace the pixels in the base map with the warped drone photo
baseMapStiched = baseMap.copy()
baseMapStiched[dronePhotoWarped > 0] = dronePhotoWarped[dronePhotoWarped > 0]

# Display the warped drone photo
plt.imshow(baseMapStiched, cmap='gray')

In [ ]:
baseMap = cv2.polylines(baseMap,[np.int32(dst)],True,0,2, cv2.LINE_AA)
draw_params = dict(matchColor = (0, 255 ,0), # draw matches in green color
 singlePointColor = None,
 matchesMask = matchesMask, # draw only inliers
 flags = 2)
 
img3 = cv2.drawMatches(droneImage, droneKeypoints, baseMap, mapKeypoints, goodMatches,None,**draw_params)
 
plt.imshow(img3, 'gray'),plt.show()

In [ ]:
# Decompose Homography to get rotation and translation

retval, rvec, tvec, n = cv2.decomposeHomographyMat(M, np.eye(3))

print("Decomposition: ", retval)
print("Rotation Vector: ", rvec[0].shape)
print("Translation Vector: ", tvec[0].shape)
print("Normal Vector: ", n[0].shape)

# Iterate over each rotation matrix and extract Euler angles
for i, R in enumerate(rvec):
    # Create a Rotation object from the rotation matrix
    rotation = Rotation.from_matrix(R)
    
    # Convert rotation to Euler angles
    euler_angles = rotation.as_euler('xyz', degrees=True)  # Assuming ZYX intrinsic rotations (roll, pitch, yaw)
    
    print(f"Rotation Matrix {i + 1}:")
    print("Roll (X-axis rotation):", euler_angles[0])
    print("Pitch (Y-axis rotation):", euler_angles[1])
    print("Yaw (Z-axis rotation):", euler_angles[2])
    print("Translation Vector:" , tvec[i][0], tvec[i][1], tvec[i][2])
    print()

# Displaying all decompositions, but these can be used to choose the correct one
# as explained above, SIFT performs the best but others can be fined tuned as well.